<a href="https://colab.research.google.com/github/Christelleelkhoury/Data-Engineering/blob/main/WORKIN~1_CEK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Working Towards Data Eng Project - Week 1**

In [3]:
!pip install dash dash-bootstrap-components pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


**Import the Libraries**

In [4]:
import dash
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

<ipython-input-4-8cf7f47a2361>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


**Load the Dataset**

For this lab, I used the **[Kaggle dataset](https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales)**, which contains information about sales of a supermarket company in 3 different branches for 3 months data. The data is formatted in a different notebook.

Columns description

*   **date**: Date of purchase (from January 2019 to March 2019)
*   **product_id**: unique IDs
*   **sales_amount**: Total amount sold
*   **store_location**: location of the supercenters (Yangon/Naypyitaw/Mandalay)



In [11]:
# Load the dataset
sales_data = pd.read_csv('https://raw.githubusercontent.com/Christelleelkhoury/Data-Engineering/refs/heads/main/data_formatted.csv')

# Display the first few rows of the dataset
sales_data.head()

,date,product_id,sales_amount,store_location
0,2019-01-05,P001,548,Yangon
1,2019-03-08,P002,80,Naypyitaw
2,2019-03-03,P003,340,Yangon
3,2019-01-27,P004,489,Yangon
4,2019-02-08,P005,634,Yangon


In [12]:
sales_data.describe()

,sales_amount
count,1000.000000
mean,322.469000
std,245.881701
min,10.000000
25%,124.250000
50%,253.500000
75%,471.250000
max,1042.000000


In [21]:
sales_data['store_location'].value_counts()

,count
store_location,
Yangon,340
Mandalay,332
Naypyitaw,328


In [20]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            1000 non-null   object
 1   product_id      1000 non-null   object
 2   sales_amount    1000 non-null   int64 
 3   store_location  1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [14]:
sales_data.nunique()

,0
date,89
product_id,1000
sales_amount,546
store_location,3


In [16]:
# Total sales
total_sales = sales_data['sales_amount'].sum()
print(f"Total Sales: {total_sales}")

Total Sales: 322469


In [17]:
# Total sales by store location
sales_by_location = sales_data.groupby('store_location')['sales_amount'].sum()
print("\nTotal Sales by Store Location:")
print(sales_by_location)


Total Sales by Store Location:
store_location
Mandalay     106032
Naypyitaw    110408
Yangon       106029
Name: sales_amount, dtype: int64


In [23]:
max_sales = sales_data['sales_amount'].max()
max_sales_row = sales_data[sales_data['sales_amount'] == max_sales]

min_sales = sales_data['sales_amount'].min()
min_sales_row = sales_data[sales_data['sales_amount'] == min_sales]

print(f"Maximum Sales Amount: {max_sales} (Product: {max_sales_row['product_id'].values[0]}, Location: {max_sales_row['store_location'].values[0]})")
print(f"Minimum Sales Amount: {min_sales} (Product: {min_sales_row['product_id'].values[0]}, Location: {min_sales_row['store_location'].values[0]})")


Maximum Sales Amount: 1042 (Product: P351, Location: Naypyitaw)
Minimum Sales Amount: 10 (Product: P823, Location: Naypyitaw)


**Set Up the Dash Application**

In [54]:
# Initialize the Dash app
app = dash.Dash(__name__)

**Create a Basic Visualization with Dash:**

In [55]:
# Aggregate sales data by store location
salesbylocation = sales_data.groupby('store_location')['sales_amount'].sum().reset_index()

In [56]:
# Define layout
app.layout = html.Div([
    html.H1("Sales Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.H3("Total Sales", style={'textAlign': 'center'}),
        html.P(f"${total_sales:,.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'fontWeight': 'bold'})
    ], style={'padding': '20px', 'border': '1px solid #ddd', 'borderRadius': '10px', 'margin': '20px', 'backgroundColor': '#f9f9f9'}),

    dcc.Graph(id='sales-bar-chart', figure=fig),

    html.Label("Select Store Location:"),
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': loc, 'value': loc} for loc in salesbylocation['store_location']],
        value=salesbylocation['store_location'].iloc[0]
    ),

    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id='date-picker',
        start_date=sales_data['date'].min(),
        end_date=sales_data['date'].max(),
        display_format='YYYY-MM-DD'
    ),

    html.Div(id='selected-location-sales', style={'marginTop': '20px', 'fontSize': '18px'}),

    dcc.Graph(id='sales-trend')
])


In [57]:
# Callback to update selected store sales
@app.callback(
    Output('selected-location-sales', 'children'),
    [Input('location-dropdown', 'value')]
)
def update_location_sales(selected_location):
    selected_sales = salesbylocation[salesbylocation['store_location'] == selected_location]['sales_amount'].values[0]
    return f"Total Sales for {selected_location}: ${selected_sales:,.2f}"

# Callback to update sales trend
@app.callback(
    Output('sales-trend', 'figure'),
    [Input('location-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_sales_trend(selected_location, start_date, end_date):
    filtered_data = sales_data[(sales_data['store_location'] == selected_location) &
                               (sales_data['date'] >= start_date) &
                               (sales_data['date'] <= end_date)]

    if filtered_data.empty:
        fig = px.bar(title='No data available for the selected criteria')
    else:
        fig = px.bar(filtered_data, x='date', y='sales_amount', title=f'Sales Trend in {selected_location}',
                      labels={'sales_amount': 'Sales Amount', 'date': 'Date'})

    return fig

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

Total sales amount is $322,469.

As shown in the bar plot where Naypitaw has the highest sales($110,408).

When analyzing the sales trend by location, we can see that the sales trend from January to March 2019. For example, in Mandalay, the highest sales occured in early march.

